# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

## Question 1: What is the sum of the outputs of the generator for limit = 5?

`sum = 8.382332347441762`

### Answer 1: `C: 8.382`

- **A**: 10.23433234744176
- **B**: 7.892332347441762
- **C**: 8.382332347441762
- **D**: 9.123332347441762

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13 # qn1: 5, qn2: 13
generator = square_root_generator(limit)
# Track the sum
sum = 0
# Store the generated values
values = []
for sqrt_value in generator:
    print(sqrt_value)
    # sum += sqrt_value
    values.append(sqrt_value)

# print(f'sum = {sum}')
print(f'13th number: {values[-1]}')

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
13th number: 3.605551275463989


## Question 2: What is the 13th number yielded

`13th item: 3.605551275463989`

### Answer 2: `B: 3.6055`

- **A**: 4.236551275463989
- **B**: 3.605551275463989
- **C**: 2.345551275463989
- **D**: 5.678551275463989

Change limit = 13

# Solution: First make sure that the following modules are installed:

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [3]:
import dlt
import duckdb

# Create a DLT pipeline for the first generator `people_1`
# The pipeline is set to load data into a DuckDB database with the dataset named 'people'
people_1_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')


# we can load any generator to a table at the pipeline destination as follows:
info = people_1_pipeline.run(people_1(),
							table_name="people",
							write_disposition="replace")

# Print metadata of the loading process for the first generator.
print(f"{info}\n\n")

# Create a second DLT pipeline for the generator `people_2`, targeting the same DuckDB database and dataset
people_2_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# Run the second pipeline, appending data from `people_2` to the existing 'people' table
info = people_2_pipeline.run(people_2(),
                             table_name="people",
                             write_disposition="append")

# Print metadata of the loading process for the second generator.
print(f"{info}\n\n")


Pipeline dlt_ipykernel_launcher load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///E:\Dev\Data Engineering\Data Engineering Bootcamp\Current\my-data-engineering-work\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1709156412.886887 is LOADED and contains no failed jobs


Pipeline dlt_ipykernel_launcher load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///E:\Dev\Data Engineering\Data Engineering Bootcamp\Current\my-data-engineering-work\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1709156413.9520924 is LOADED and contains no failed jobs




In [4]:
# show outcome

# Connect to the DuckDB database created by the first generator
conn = duckdb.connect(f"{people_1_pipeline.pipeline_name}.duckdb")

# Setting the search path to the dataset 'people' and displaying available tables
conn.sql(f"SET search_path = '{people_1_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n table below:")

# Fetching the appended data from the 'people' table and displaying it
people = conn.sql("SELECT * FROM people").df()
display(people)

# As you can see, the same data was loaded in both cases.

# Calculate the sum of ages from the combined data of `people_1` and `people_2` in the 'people' table
sum_of_ages_p1_p2 = conn.execute("SELECT SUM(age) FROM people").fetchone()[0]
print(f"\n\nSum of ages from generators `people_1()` and `people_2()` combined: {sum_of_ages_p1_p2}")

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1709156412.886887,24S3bcMJm4a8HQ,None
1,2,Person_2,27,City_A,1709156412.886887,DQbOVsuPLSTr3w,None
2,3,Person_3,28,City_A,1709156412.886887,PYmJnRWiIa1R+g,None
3,4,Person_4,29,City_A,1709156412.886887,JyRez7uZytErog,None
4,5,Person_5,30,City_A,1709156412.886887,heHbRrFu9N8Axw,None
5,3,Person_3,33,City_B,1709156413.9520924,FGgs3r42P/OKPQ,Job_3
6,4,Person_4,34,City_B,1709156413.9520924,FmJO9QL9nigHdA,Job_4
7,5,Person_5,35,City_B,1709156413.9520924,kXyRIUljdmemzQ,Job_5
8,6,Person_6,36,City_B,1709156413.9520924,SjeBgdNLuK6zaA,Job_6
9,7,Person_7,37,City_B,1709156413.9520924,QvmEXcYMHGSj1A,Job_7




Sum of ages from generators `people_1()` and `people_2()` combined: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [5]:
import dlt
import duckdb

# Create a DLT pipeline for the first generator `people_1`
# The pipeline is set to load data into a DuckDB database with the dataset named 'people'
people_1_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')


# we can load any generator to a table at the pipeline destination as follows:
info = people_1_pipeline.run(people_1(),
							table_name="people_merged",
							write_disposition="replace",
							primary_key="ID")

# Print metadata of the loading process for the first generator.
print(f"{info}\n\n")

# Create a second DLT pipeline for the generator `people_2`, targeting the same DuckDB database and dataset
people_2_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# Run the second pipeline, appending data from `people_2` to the existing 'people' table
info = people_2_pipeline.run(people_2(),
                             table_name="people_merged",
                             write_disposition="merge",
                             primary_key="ID")

# Print metadata of the loading process for the second generator.
print(f"{info}\n\n")

# Connect to the DuckDB database created by the first generator
conn = duckdb.connect(f"{people_1_pipeline.pipeline_name}.duckdb")

# Setting the search path to the dataset 'people' and displaying available tables
conn.sql(f"SET search_path = '{people_1_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# Display the merged data from the 'people_merged' table.
print("\n\n\nData from the 'people_merged' table:")
people = conn.sql("SELECT * FROM people_merged").df()
display(people)

# As you can see, the same data was loaded in both cases.

# Calculate the sum of ages from the merged data of `people_1` and `people_2` in the 'people_merged' table
sum_of_ages_p1_p2 = conn.execute("SELECT SUM(age) FROM people_merged").fetchone()[0]
print(f"\n\nSum of ages from generators `people_1()` merged with generator `people_2()` is: {sum_of_ages_p1_p2}")


Pipeline dlt_ipykernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///E:\Dev\Data Engineering\Data Engineering Bootcamp\Current\my-data-engineering-work\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1709156415.0160508 is LOADED and contains no failed jobs


Pipeline dlt_ipykernel_launcher load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///E:\Dev\Data Engineering\Data Engineering Bootcamp\Current\my-data-engineering-work\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1709156415.7862613 is LOADED and contains no failed jobs


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merged       │
└─────────────────────┘




Data from the 'people_merged' table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1709156415.0160508,J40/sleUE66QDg,None
1,2,Person_2,27,City_A,1709156415.0160508,u8vUjdW4z0OnZg,None
2,8,Person_8,38,City_B,1709156415.7862613,0eMcQfadT8ilZg,Job_8
3,4,Person_4,34,City_B,1709156415.7862613,yrhgrR9ivYT3Mg,Job_4
4,5,Person_5,35,City_B,1709156415.7862613,XndDSHZORqlkLA,Job_5
5,3,Person_3,33,City_B,1709156415.7862613,TWFY9iDAFVkl7A,Job_3
6,6,Person_6,36,City_B,1709156415.7862613,ZKk2xSUj68CrZA,Job_6
7,7,Person_7,37,City_B,1709156415.7862613,uyRO7wi3/7/CdA,Job_7




Sum of ages from generators `people_1()` merged with generator `people_2()` is: 266


## Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

### Answer 3: `A: 353`

- **A**: 353
- **B**: 365
- **C**: 378
- **D**: 390

## Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

### Answer 4: `B: 266`

- **A**: 215
- **B**: 266
- **C**: 241
- **D**: 258